# Classifier evaluation

In the last two sessions, we implemented our first prediction models (T05: linear and logistic regression, T06: decision trees) -- hurray! We already separated the dataset features from their class, fed these features to a regression model or decision tree and trained the model to make a prediction, whether a patient is ill or not.

We can say that we model described the existing data to a good extent (if we hadn't restricted the `max_depth` of the tree, our model would yield to a 100% accuracy). But did our algorithm actually "learn" anything? In other words, would our linear/tree model predict the class correctly if we give it a completely new patient and his or her health records? We don't know yet.

We trained and tested our decision tree model's accuracy on the same data for a simplicity reason, but that's not usually the case in real machine learning projects. Usually, we want to create algorithms that generalize beyond the observed data. It makes sense that we could only evaluate the ability of the model to generalize from a data sample using data that it had not seen before.

## Train-Test split

Now we understand that we would want some additional, unseen before data entries to test our prediction model, but where do we get them? Well, meaningful data collection is hard, so why don't we just say from the beginning that we preserve some part of the existing dataset and use it later for evaluation purpose. Sounds good, right?

Splitting your dataset is essential for an unbiased evaluation of prediction performance. In most cases, it’s enough to split your dataset randomly into two subsets:

1.   The *training set* is applied to train, or fit, your model.
2.   The *test set* is needed for an unbiased evaluation of the final model.


In [3]:
import pandas as pd
import numpy as np

In [ ]:
# Data preparation (from last session)

df = pd.read_csv('data/data_banknote_authentication.txt', names=['Variance', 'Skewness', 'Kurtosis', 'Entropy', 'Class'])

print(df.shape)
print(df.columns)

# separate target (dependent) variable from features (independent variables)
X = df.drop('Class', axis=1).copy()
y = df['Class'].copy()

**Recap:** If you are curious how these features were extracted from photos of the banknotes, check the [Wavelet transform](https://towardsdatascience.com/what-is-wavelet-and-how-we-use-it-for-data-science-d19427699cef) out. But simply speaking we convert each image into a distribution of its pixels and extract some statistics about the distribution:

*  Variance - measure of variability or spread of the distribution;
*   Skewness - measure of symmetry of a distribution. In normal distributions skewness = 0. Distribution skewed to the right  (a.k.a positive skewed) will have a longer right tail and to the left (negative skew) a longer left tail (longer tails occur because of outliers);
*   Kurtosis - measures how "thick" are the tails of the distribution;
* Entropy - measure of uncertainty.

But don't get confused by the features - that's not the important thing in this exercise.

We also have a fifth column `Class`. That looks like the target, that we want to predict. Indeed, class 1 stands for "genuine banknote" and class 0 for "fake banknote".


### Exercise: Make a training and test data split (see T05)

While training the logistic regression model in T05, we already made a training and test data split. Please apply this step here.

In [1]:
# todo


### Exercise: Evaluating the Algorithm
Now, train a decision tree model on the training data and evaluate it using test data. Obtain a confusion matrix and compute performance metrics.

In [ ]:
# todo 